In [59]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import datetime

In [65]:
traffic_raw = []
r = requests.get("https://data.austintexas.gov/resource/x44q-icha.json?$where=origin_reader_identifier like '%amar_%'"
                 , params = {
#                "$limit": 10,
                "$limit": 300000,
                "$offset": 0
                
                })
if r.status_code == 200:     
        traffic_raw = r.json()
        #print("Record Start = " + str(offset) + "Pagination is " + str(increment))
        #offset += increment
        
else:
    print("***** IT IS BROKEN *****")
    print(r.status_code)

In [66]:
travel_data = [{
    "ID": rec["record_id"],
    "Start": rec["origin_reader_identifier"].upper(),
    "Ending": rec["destination_reader_identifier"].upper(),
    "TravelTime": int(rec["travel_time_seconds"]),
    "MPH": int(rec["speed_miles_per_hour"]),
    "StartTime": rec["start_time"],
    "EndTime": rec["end_time"]
#     "Year": pd.to_datetime(rec["traffic_report_status_date_time"]).strftime("%Y"),
#     "Hour": pd.to_datetime(rec["traffic_report_status_date_time"]).hour
} for rec in traffic_raw]

In [67]:
td = pd.DataFrame(travel_data)
td

,ID,Start,Ending,TravelTime,MPH,StartTime,EndTime
0,c152019be9e39d8a087757fd2e77bbbc,LAMAR_51ST,LAMAR_45TH,247,8,2017-09-07T09:38:42,2017-09-07T09:42:49
1,a78be151a3a70748815be0af6019a01a,LAMAR_51ST,LAMAR_45TH,73,28,2017-09-07T09:42:40,2017-09-07T09:43:53
2,c457c35ca78bf7063681a3dbd8efbcd4,LAMAR_51ST,LAMAR_45TH,68,30,2017-09-07T09:43:21,2017-09-07T09:44:28
3,1b05d1c5f70d930ed8905e356ef0f00e,LAMAR_51ST,LAMAR_45TH,150,14,2017-09-07T09:43:19,2017-09-07T09:45:48
4,7945973ad9b48714e6635b83a5dbba09,LAMAR_51ST,LAMAR_45TH,132,15,2017-09-07T09:43:27,2017-09-07T09:45:39
...,...,...,...,...,...,...,...
299995,b46139ac1a4044bc1ce7239f95c03845,LAMAR_6TH,LAMAR_5TH,77,4,2017-09-19T20:38:39,2017-09-19T20:39:55
299996,56d17558762a2465447930a319f55560,LAMAR_6TH,LAMAR_5TH,20,17,2017-09-19T20:40:23,2017-09-19T20:40:42
299997,285e2a992d351a54c3c139c95402164f,LAMAR_6TH,LAMAR_5TH,83,4,2017-09-19T20:42:20,2017-09-19T20:43:43
299998,36ef1e9598cf64804e2bff6b5f90e515,LAMAR_6TH,LAMAR_5TH,19,18,2017-09-19T20:43:28,2017-09-19T20:43:47


In [68]:
pattern = ("(LAMAR)")
td["hwy"] = td["Ending"].str.extract("(LAMAR)", expand = True)
td["RecordDate"] = td["StartTime"].str.extract("(\d+-\d+-\d+)", expand = True)
filtered_df = td[td.hwy.notnull()]

In [71]:
travel_by_date = pd.DataFrame(filtered_df.groupby("RecordDate").agg({
                                 "ID" : "count",
                                 "TravelTime" : "mean",
                                 "MPH" : "mean"
                                })).rename(columns={
                                                        "ID" : "Vehicle Count",
                                                        "TravelTime" : "AVG Travel Time",
                                                        "MPH" : "AVG Speed (mph)"
                                                    })
travel_by_date

,Vehicle Count,AVG Travel Time,AVG Speed (mph)
RecordDate,,,
2017-09-07,5410,173.600000,22.602403
2017-09-08,28350,165.611464,22.339189
2017-09-09,23801,149.093652,24.056720
2017-09-10,18626,137.560346,25.574036
2017-09-11,25190,154.084518,23.662048
2017-09-12,26539,164.339576,22.780323
2017-09-13,26023,164.978481,22.800254
2017-09-14,27582,160.725654,23.071677
2017-09-15,28632,160.765821,23.057872


In [ ]:
travel_by_date